# Appending WRF data to the ldeo-glaciology bucket without a cluster

This NB demonstrates the issue raised in [this issue](https://github.com/pydata/xarray/issues/5878), but using real WRF data. 


In [2]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
from glob import glob
import os

import gcsfs
from tqdm import tqdm
import fsspec
import xarray as xr
xr.set_options(display_style="html");
import json
import pandas as pd


### list the netcdf files

In [3]:
fs = gcsfs.GCSFileSystem(project='ldeo-glaciology', mode='ab', cache_timeout = 0)

In [4]:
gcsdir = 'gs://ldeo-glaciology'
ampsdir = 'AMPS'
amps_ver = 'WRF_24'
domain = 'domain_03'
filepattern = 'wrfout_d03_20190930*'
pattern = os.path.join(gcsdir, ampsdir, amps_ver, domain, filepattern)
NCs = fs.glob(pattern)
print(f"Total of {len(NCs)} wrf files.\n")
NCs

Total of 8 wrf files.



['ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f003.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f006.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f009.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093000_f012.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f003.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f006.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f009.nc',
 'ldeo-glaciology/AMPS/WRF_24/domain_03/wrfout_d03_2019093012_f012.nc']

### load two netcdfs

In [6]:
ds = []
openfile = fs.open(f'gs://{NCs[0]}', mode='rb') 
ds1 = xr.open_dataset(openfile, engine='h5netcdf', chunks={'south_north': -1, 'west_east': -1})
openfile = fs.open(f'gs://{NCs[1]}', mode='rb') 
ds2 = xr.open_dataset(openfile, engine='h5netcdf', chunks={'south_north': -1, 'west_east': -1})

### remove some of the variables

In [7]:
vars_save = [
#     'U', 'V', 'W', 'PH', 'PHB', 'T', 'P', 'PB', 'P_HYD',
             'Q2', 'T2', 'PSFC', 'U10', 'V10',
#              'QVAPOR', 'QICE', 'QCLOUD', 'QRAIN', 'QSNOW', 
             'LANDMASK', 'HGT',
             'SST', 'SSTSK', 'TSK',
             'RAINC', 'RAINSH', 'RAINNC', 
             'QFX', 'HFX', 'LH', 'PBLH', 'ACSNOM', 'SWDOWN', 'LWDNB', 'GLW',
            'SNOW', 'SNOWH'
]

In [8]:
ds1_stripped = ds1[vars_save]
ds2_stripped = ds2[vars_save]
ds1_stripped_rechunked = ds1_stripped.chunk({"Time": -1})   # doesnt do anything because the ds in only 1 long in the time dimension
ds2_stripped_rechunked = ds2_stripped.chunk({"Time": -1})   # doesnt do anything because the ds in only 1 long in the time dimension
print(f'stripped ds is {ds1_stripped_rechunked.nbytes/1e9} Gb')
print(f'Original ds was {ds1.nbytes/1e9} Gb')

stripped ds is 0.069862508 Gb
Original ds was 2.680378751 Gb


### compute the results (does not need a cluster because they are small)

In [9]:
ds1_stripped_rechunked.compute()
ds2_stripped_rechunked.compute()

<xarray.Dataset>
Dimensions:   (Time: 1, south_north: 1035, west_east: 675)
Coordinates:
    XLAT      (Time, south_north, west_east) float32 -80.53 -80.55 ... -67.92
    XLONG     (Time, south_north, west_east) float32 59.73 59.66 ... -157.9
    XTIME     (Time) datetime64[ns] 2019-09-30T06:00:00
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    Q2        (Time, south_north, west_east) float32 9.95e-06 ... 0.000965
    T2        (Time, south_north, west_east) float32 212.5 212.4 ... 257.3 257.3
    PSFC      (Time, south_north, west_east) float32 5.912e+04 ... 9.761e+04
    U10       (Time, south_north, west_east) float32 2.907 2.909 ... 10.31 10.26
    V10       (Time, south_north, west_east) float32 -2.874 -2.834 ... -2.795
    LANDMASK  (Time, south_north, west_east) float32 1.0 1.0 1.0 ... 1.0 1.0 1.0
    HGT       (Time, south_north, west_east) float32 3.791e+03 3.792e+03 ... 0.0
    SST       (Time, south_north, west_east) float32 271.4 271.4 ... 271.4 271.4
    SSTSK     (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    TSK       (Time, south_north, west_east) float32 212.2 212.1 ... 257.0 257.0
    RAINC     (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RAINSH    (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    RAINNC    (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    QFX       (Time, south_north, west_east) float32 1.554e-09 ... 2.954e-06
    HFX       (Time, south_north, west_east) float32 -2.234 -2.24 ... -10.46
    LH        (Time, south_north, west_east) float32 0.004399 0.004321 ... 6.338
    PBLH      (Time, south_north, west_east) float32 49.66 49.64 ... 753.4 753.5
    ACSNOM    (Time, south_north, west_east) float32 0.0 0.0 0.0 ... 0.0 0.0 0.0
    SWDOWN    (Time, south_north, west_east) float32 246.3 245.7 ... 0.0 0.0
    LWDNB     (Time, south_north, west_east) float32 67.42 67.26 ... 182.4 185.1
    GLW       (Time, south_north, west_east) float32 67.42 67.26 ... 182.4 185.1
    SNOW      (Time, south_north, west_east) float32 100.0 100.0 ... 10.04 10.04
    SNOWH     (Time, south_north, west_east) float32 1.108 1.108 ... 0.05019
Attributes:
    TITLE:                            OUTPUT FROM WRF V3.9.1.1 MODEL
    START_DATE:                      2019-09-30_00:00:00
    SIMULATION_START_DATE:           2019-09-30_00:00:00
    WEST-EAST_GRID_DIMENSION:        [676]
    SOUTH-NORTH_GRID_DIMENSION:      [1036]
    BOTTOM-TOP_GRID_DIMENSION:       [61]
    DX:                              [2666.667]
    DY:                              [2666.667]
    SKEBS_ON:                        [0]
    SPEC_BDY_FINAL_MU:               [1]
    USE_Q_DIABATIC:                  [0]
    GRIDTYPE:                        C
    DIFF_OPT:                        [1]
    KM_OPT:                          [4]
    DAMP_OPT:                        [3]
    DAMPCOEF:                        [0.21]
    KHDIF:                           [0.]
    KVDIF:                           [0.]
    MP_PHYSICS:                      [4]
    RA_LW_PHYSICS:                   [4]
    RA_SW_PHYSICS:                   [2]
    SF_SFCLAY_PHYSICS:               [2]
    SF_SURFACE_PHYSICS:              [2]
    BL_PBL_PHYSICS:                  [2]
    CU_PHYSICS:                      [0]
    SF_LAKE_PHYSICS:                 [0]
    SURFACE_INPUT_SOURCE:            [1]
    SST_UPDATE:                      [0]
    GRID_FDDA:                       [0]
    GFDDA_INTERVAL_M:                [0]
    GFDDA_END_H:                     [0]
    GRID_SFDDA:                      [0]
    SGFDDA_INTERVAL_M:               [0]
    SGFDDA_END_H:                    [0]
    HYPSOMETRIC_OPT:                 [2]
    USE_THETA_M:                     [0]
    GWD_OPT:                         [0]
    SF_URBAN_PHYSICS:                [0]
    SF_OCEAN_PHYSICS:                [0]
    SHCU_PHYSICS:                    [0]
    MFSHCONV:                        [0]
    FEEDBACK: 

### write the first ds

In [10]:
outdir = os.path.join(gcsdir, ampsdir, amps_ver, domain, 'zarr-raw/')

with open('../secrets/ldeo-glaciology-bc97b12df06b.json') as token_file:
    token = json.load(token_file)
amps_mapper = fsspec.get_mapper(outdir + 'wrfout_d03_201909_2D_rechunked_appendingTest5.zarr', mode='a',
                            token=token)

ds1_stripped_rechunked.to_zarr(amps_mapper, 
                    mode='w', 
                    consolidated='True');

### append the 2nd ds

In [11]:
ds2_stripped_rechunked.to_zarr(amps_mapper, mode="a",
           consolidated=True,
           append_dim="Time")

## load the results and see how many time steps it has

In [12]:
both_stripped_rechunked_reloaded = xr.open_zarr(amps_mapper)
both_stripped_rechunked_reloaded

<xarray.Dataset>
Dimensions:   (Time: 1, south_north: 1035, west_east: 675)
Coordinates:
    XLAT      (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    XLONG     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    XTIME     (Time) datetime64[ns] dask.array<chunksize=(1,), meta=np.ndarray>
Dimensions without coordinates: Time, south_north, west_east
Data variables:
    ACSNOM    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    GLW       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    HFX       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    HGT       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    LANDMASK  (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    LH        (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    LWDNB     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    PBLH      (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    PSFC      (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    Q2        (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    QFX       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    RAINC     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    RAINNC    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    RAINSH    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    SNOW      (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    SNOWH     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    SST       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    SSTSK     (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    SWDOWN    (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    T2        (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    TSK       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    U10       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
    V10       (Time, south_north, west_east) float32 dask.array<chunksize=(1, 1035, 675), meta=np.ndarray>
Attributes:
    ADAPT_DT_MAX:                    [17.0]
    ADAPT_DT_MIN:                    [8.0]
    ADAPT_DT_START:                  [9.0]
    AER_ANGEXP_OPT:                  [1]
    AER_ANGEXP_VAL:                  [1.2999999523162842]
    AER_AOD550_OPT:                  [1]
    AER_AOD550_VAL:                  [0.11999999731779099]
    AER_ASY_OPT:                     [1]
    AER_ASY_VAL:                     [0.8999999761581421]
    AER_OPT:                         [0]
    AER_SSA_OPT:                     [1]
    AER_SSA_VAL:                     [0.8500000238418579]
    AER_TYPE:                        [1]
    BLDT:                            [0.0]
    BL_PBL_PHYSICS:                  [2]
    BOTTOM-TOP_GRID_DIMENSION:       [61]
    BOTTOM-TOP_PATCH_END_STAG:       [61]
    BOTTOM-TOP_PATCH_END_UNSTAG:     [60]
    BOTTOM-TOP_PATCH_START_STAG:     [1]
    BOTTOM-TOP_PATCH_START_UNSTAG:   [1]
    BUCKET_J:                        [-1.0]
    BUCKET_MM:                       [-1.0]
    CEN_LAT:                         [-82.04856872558594]
    CEN_LON:        

### loading the XTIME coordinates of the reloaded zarr and the original two datasets suggests that the second one is not getting written

In [76]:
print(both_stripped_rechunked_reloaded.XTIME.values)
print(ds1_stripped_rechunked.XTIME.values)
print(ds2_stripped_rechunked.XTIME.values)


['2019-09-30T03:00:00.000000000']
['2019-09-30T03:00:00.000000000']
['2019-09-30T06:00:00.000000000']
